In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         60           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         88           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        401           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        162           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1449           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      15195            True  
house-pr

In [ ]:
import os
import logging
import numpy as np 
import pandas as pd
import time
import gc
import shutil
import time
import warnings
import optuna
import datatable as dt
import ftplib
from concurrent.futures import ThreadPoolExecutor
import threading
import random
import string

from sklearn.model_selection import *
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import *
from sklearn.metrics import *


from tqdm.notebook import tqdm
from copy import deepcopy

from lightgbm import *
from catboost import *
from xgboost import *


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [ ]:
# Work in PC or Kaggle
IN_PC = ('VSCODE_CWD' in os.environ) and (not 'AK_KAGGLE_ENV' in os.environ)
IN_DOCKER = ('AK_KAGGLE_ENV' in os.environ)
IN_COLAB = ('COLAB_GPU' in os.environ)
# DATA_PATH = "../../data" if os.path.exists("../../data") and IN_PC else "../../../data" if os.path.exists("../../../data") and IN_PC else "/kaggle"

if IN_PC:
    DATA_PATH = "../../data" if os.path.isdir("../../data") else "../../../data"
elif(IN_COLAB):
    DATA_PATH = "./drive/MyDrive/Colab_Kaggle/data"
else:
    DATA_PATH = "/kaggle"
    

for dirname, _, filenames in os.walk(DATA_PATH + '/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# shutil.copy(DATA_PATH + '/input/00000-01/fix.db', DATA_PATH + '/working/fix.db')

./drive/MyDrive/Colab_Kaggle/data/input/tabular-playground-series-sep-2021/sample_solution.csv
./drive/MyDrive/Colab_Kaggle/data/input/tabular-playground-series-sep-2021/test.csv
./drive/MyDrive/Colab_Kaggle/data/input/tabular-playground-series-sep-2021/train.csv


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Oct 29 14:34:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [13]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
n_splits = 5
early_stopping=5
time_start = time.time()
# time_limit=31500
time_limit=40
GPU=False
# GPU=True
# no_stop=False
no_stop=True
# ak_n_trials = 1700 # for n_trials optuna
ak_n_trials = 10 # for n_trials optuna # هیچوقت برابر یک نگذار
study_lst=dict()
folder_ftp='tps1021'

competition='tps1021'
min_accuracy= 0.84

# config = {'name': 'DEMO', 'model': 'XGBClassifier','csv_name': competition+'_xgb_parameters.csv', 'db_name': competition+'_xgb_study_data.db'}

config = {'name': 'XGB', 'model': 'XGBClassifier','csv_name': competition+'_xgb_parameters.csv', 'db_name': competition+'_xgb_study_data.db'}
# config = {'name': 'XGB', 'model': 'XGBRegressor','csv_name': competition+'_xgb_parameters.csv', 'db_name': competition+'_xgb_study_data.db'}
# # ####============#########
# config = {'name': 'LGBM', 'model': 'LGBMClassifier','csv_name': competition+'_lgbm_parameters.csv', 'db_name': competition+'_lgbm_study_data.db'}
# config = {'name': 'LGBM', 'model': 'LGBMRegressor','csv_name': competition+'_lgbm_parameters.csv', 'db_name': competition+'_lgbm_study_data.db'}
# # ####============#########
# config = {'name': 'CAT', 'model': 'CatBoostClassifier','csv_name': competition+'_cat_parameters.csv', 'db_name': competition+'_cat_study_data.db'}
# config = {'name': 'CAT', 'model': 'CatBoostRegressor','csv_name': competition+'_cat_parameters.csv', 'db_name': competition+'_cat_study_data.db'}



# csv_name='xgboost_parameters.csv'

In [ ]:
# %%time
# https://www.thepythoncode.com/article/download-and-upload-files-in-ftp-server-using-python
# https://www.tutorialspoint.com/python_network_programming/python_ftp.htm
# https://stackoverflow.com/a/15170571
# https://stackoverflow.com/questions/51502812/why-doesnt-concurrent-futures-threadpoolexecutor-submit-return-immediately

# ftp.storbinary(f"STOR {filename}", file)
# ftp.retrbinary(f"RETR {filename}", file.write)
# random.choice(string.ascii_letters,k=6)
# np.random.choice(string.ascii_lowercase)
def ftp_ok():
    try:
        ftp_1.voidcmd("NOOP")
        return ftp_1
    except:
        host = "94.23.57.47"
        user = "kaggle_6u9kgd2"
        port = 21
        password = "6K!UsDyI30I!GML#C7i2Lm8F3u"
        ftp_1 = ftplib.FTP(host,user,password)
        ftp_1.encoding = "utf-8"
        return ftp_1
        
    
               
def send_file(filename,ftp,to_folder, verbose=False):
#     ftp =ftp_ok()
    try:
        ftp.cwd(to_folder)
        ftp.storbinary(f"STOR {filename}" ,open(filename, 'rb'))
        if verbose: print("Sended: {0}".format(filename))
    except BaseException as be:
        Err = be
        # print ("Error-2: {0} ".format(be))
        
def get_file(filename,ftp,to_folder,verbose=False):
#     ftp =ftp_ok()
    try:
        ftp.cwd(to_folder)
        ftp.retrbinary(f"RETR {filename}" ,open(filename, 'wb').write)
        if verbose: print("Geted: {0}".format(filename))
    except BaseException as be:
        Err = be
        # print ("Error-1: {0} ".format(be))
        
def send_file_tqdm(filename,ftp):
    try:
        with open(filename, 'wb') as fd:
            total = ftp.size(filename)

            with tqdm(total=total,
                      unit='B', unit_scale=True, unit_divisor=1024,) as pbar:
                def cb(data):
                    pbar.update(len(data))
                    fd.write(data)

                ftp.retrbinary('RETR {}'.format(filename), cb)
    except BaseException as be:
        Err = be
#         print ("Error: {0} ".format(be))

def upload_files():
    host = "94.23.57.47"
    user = "kaggle_6u9kgd2"
    port = 21
    password = "6K!UsDyI30I!GML#C7i2Lm8F3u"
    ftp = dict()

    for n in range(10):
        ftp[n] = ftplib.FTP(host,user,password)
        ftp[n].encoding = "utf-8"

    while True:
        time.sleep(1)
        with ThreadPoolExecutor(max_workers=10) as executor:
            #         print('|||'*60,'\n')
            files = [name for name in os.listdir(DATA_PATH + '/working/') if not name.startswith('__') and not name in ('custom.css')]
            ftp_files = []

            ftp[0].close()
    #         ftp[0].quit()
            ftp[0] = ftplib.FTP(host,user,password)
    #         ftp[0].login(user=user, passwd = password)
            ftp[0].cwd(folder_ftp)

            ftp[0].dir(ftp_files.append)
            ftp_files_lst = [ftp_files[i].split() for i in range(len(ftp_files))]
            # print(ftp_files_lst)
            #     print(dd[p].split()[4])
            #         ftp_files_lst = ftp_files.split()


            for b in range(len(ftp_files_lst)):
                for a in range(len(files)):
            #         if ftp_files_lst[b][8] == files[a]:
            #         print((int(ftp_files_lst[b][4])))
            #         print((os.path.getsize(files[a])))
            #         print()
            #         if int(ftp_files_lst[b][4]) == os.path.getsize(files[a]):
            #             print('yyyyyy')
                    if ftp_files_lst[b][8] == files[a] and int(ftp_files_lst[b][4]) == os.path.getsize(files[a]):
                        files.remove(files[a])
                        break;
    #                     print('ddd')

            if not len(files) == 0:
                for n in range(10):
                    if n < len(files)  and os.path.exists(files[n]):
                #     ftp[n] = ftplib.FTP(host,user,password)
                #     ftp[n].encoding = "utf-8"
                #             time.sleep(1)
    #                         print('=='*60,files[n],'\n')
                        try:
                            executor.submit(send_file, files[n],ftp[n],folder_ftp) 
                        except BaseException as be:
                            Err = be
                            print ("Error-3: {0}: ".format(be))
                #         print('=='*60,'\n')

                #     executor.map(run_study, range(45))
            
x = threading.Thread(target=upload_files).start()
# x.start()
# xs = ThreadPoolExecutor(max_workers=10)
# xs.submit(upload_files)

In [ ]:
# %%time
if config['name'] == 'XGB':
    if os.path.isfile(DATA_PATH + '/working/' + config['csv_name']):
        parameters_csv = pd.read_csv(DATA_PATH + '/working/' + config['csv_name'], dtype='str')
    else:
        parameters_csv = pd.read_csv(DATA_PATH + '/working/empty_xgb_parameters.csv')
        
elif config['name'] == 'LGBM':
    if os.path.isfile(DATA_PATH + '/working/' + config['csv_name']):
        parameters_csv = pd.read_csv(DATA_PATH + '/working/' + config['csv_name'], dtype='str')
    else:
        parameters_csv = pd.read_csv(DATA_PATH + '/working/empty_lgbm_parameters.csv')
        
elif config['name'] == 'CAT':
    if os.path.isfile(DATA_PATH + '/working/' + config['csv_name']):
        parameters_csv = pd.read_csv(DATA_PATH + '/working/' + config['csv_name'], dtype='str')
    else:
        parameters_csv = pd.read_csv(DATA_PATH + '/working/empty_cat_parameters.csv')

In [ ]:
X_train = pd.read_hdf(DATA_PATH + '/input/akimport/X_train.h5',key='df', mode='r')
X_test = pd.read_hdf(DATA_PATH + '/input/akimport/X_test.h5',key='df', mode='r')
y_train = pd.read_hdf(DATA_PATH + '/input/akimport/y_train.h5',key='df', mode='r')
y_test = pd.read_hdf(DATA_PATH + '/input/akimport/y_test.h5',key='df', mode='r')

In [ ]:
# X_train = pd.read_pickle(DATA_PATH +'/input/ak-import/X_train.pkl.gzip')
# X_test = pd.read_pickle(DATA_PATH +'/input/ak-import/X_test.pkl.gzip')
# y_train = pd.read_pickle(DATA_PATH +'/input/ak-import/y_train.pkl.gzip')
# y_test = pd.read_pickle(DATA_PATH +'/input/ak-import/y_test.pkl.gzip')

In [ ]:
def xgb_objective(trial):
    global parameters_csv
    xgb_params__ = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'gpu_hist',
        'predictor': "gpu_predictor",
        # start
        "verbosity": 0, # Valid values of 0 (silent), 1 (warning), 2 (info), and 3 (debug).
#         "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "booster": trial.suggest_categorical("booster", ["gbtree"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True), # L2 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True), # L1 regularization weight.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0), # sampling ratio for training data.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0), # sampling according to each tree.
        # akmeghdad
        "colsample_bylevel": trial.suggest_float("colsample_bytree", 0.2, 1.0), # sampling according to each tree.
        'n_estimators': trial.suggest_int('n_estimators', 400, 4000, step=100),
        'use_label_encoder': trial.suggest_categorical("use_label_encoder", ["True", "False"]),
        'random_state': 59
        
        # 'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
    }
    
    xgb_params = {   'max_depth': 6,
    'n_estimators': 9500,
    'subsample': 0.7,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.6000000000000001,
    'min_child_weight': 56.41980735551558,
    'reg_lambda': 75.56651890088857,
    'reg_alpha': 0.11766857055687065,
    'gamma': 0.6407823221122686,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
                 }
    
    if not GPU :
        del xgb_params['tree_method']
        del xgb_params['predictor']
#         del xgb_params['booster']
#         xgb_params['gpu_id'] = -1


    if xgb_params["booster"] in ["gbtree", "dart"]:
        # xgb_params["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        xgb_params["max_depth"] = trial.suggest_int("max_depth", 1, 9, step=1)
        xgb_params["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        xgb_params["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True) # learning_rate
        xgb_params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        xgb_params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if xgb_params["booster"] == "dart":
        xgb_params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        xgb_params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        xgb_params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        xgb_params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
        
    xgb_no_gpu_params={
        'tree_method': '',
        'predictor': "",
    }

    # model = config['model'](**xgb_params)
    model = eval(config['model'])(**xgb_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
#         eval_metric=["error", "logloss", "auc"],
#         verbose=False,
        verbose=100,
        early_stopping_rounds = early_stopping
        )
    y_predicted = model.predict_proba(X_test)
    accuracy = roc_auc_score(y_test, y_predicted[:, 1])
    # accuracy = accuracy_score(y_test, y_predicted)
    # print('Accuracy:', round(accuracy_score(y_test, y_predicted[:, 1]) * 100,2),'%')
    
    if accuracy > min_accuracy:
        xgb_params['score'] = accuracy
        xgb_params['model'] = config['model']
        xgb_params['date'] = time.strftime('%Y-%m-%d %H:%M:%S')
        err,ok1,ok2 = 0,0,0
        id = parameters_csv.shape[0]+1

        for i, k in xgb_params.items():
            if i in parameters_csv.columns:
                parameters_csv.loc[id, i] = k
                ok1 += 1
            elif '+'+i in parameters_csv.columns:
                parameters_csv.loc[id, '+'+i] = k
                ok2 += 1
    
        parameters_csv.to_csv(config['csv_name'], index=False,  float_format='%g')
        parameters_csv = pd.read_csv(config['csv_name'], dtype='str')
        parameters_csv = parameters_csv.drop_duplicates(subset=parameters_csv.columns[2:].to_list())
        parameters_csv.to_csv(config['csv_name'], index=False,  float_format='%g')

#     print("============ Accuracy: %.4f%% ==============" % (accuracy * 100.0))
    return accuracy

In [ ]:
def lgbm_objective(trial):
    global parameters_csv
    lgbm_params = {
        "device_type" : "gpu",
        "objective": "binary",
        # "metric": "binary_logloss",
        "metric": "auc",
        "verbosity": -1, # < 0: Fatal, = 0: Error (Warning), = 1: Info, > 1: Debug
        "boosting_type": "gbdt", # gbdt, rf, dart, goss
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),# 0.1<-<1.0
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),# 0.1<-<1.0
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        # akmeghdad
        # https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5
        "n_estimators": trial.suggest_int("n_estimators", 400, 10000, step=100),
        # "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1),
        # "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95, step=0.1),
        "max_depth":-1,
        "random_state": 59,
        # "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        
            # 'class_weight': None,
            # 'min_child_weight': 0.001,
            # 'min_split_gain': 0.0,
            # 'n_jobs': -1,
            # 'reg_alpha': 0.0,
            # 'reg_lambda': 0.0,
            # 'silent': True,
            # 'subsample': 1.0,
            # 'subsample_for_bin': 200000,
            # 'subsample_freq': 0
        
    }
    
    if not GPU :
        del lgbm_params['device_type']

    model = eval(config['model'])(**lgbm_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
#         eval_metric=["error", "logloss", "auc"],
#         verbose=False,
        verbose=100,
        early_stopping_rounds = early_stopping
        )
    y_predicted = model.predict_proba(X_test)
    accuracy = roc_auc_score(y_test, y_predicted[:, 1])
    # accuracy = accuracy_score(y_test, y_predicted)
    # print('Accuracy:', round(accuracy_score(y_test, y_predicted[:, 1]) * 100,2),'%')
    
    if accuracy > min_accuracy:
        lgbm_params['score'] = accuracy
        lgbm_params['model'] = config['model']
        lgbm_params['date'] = time.strftime('%Y-%m-%d %H:%M:%S')
        err,ok1,ok2 = 0,0,0
        id = parameters_csv.shape[0]+1

        for i, k in lgbm_params.items():
            if i in parameters_csv.columns:
                parameters_csv.loc[id, i] = k
                ok1 += 1
            elif '+'+i in parameters_csv.columns:
                parameters_csv.loc[id, '+'+i] = k
                ok2 += 1
    
        parameters_csv.to_csv(config['csv_name'], index=False,  float_format='%g')
        parameters_csv = pd.read_csv(config['csv_name'], dtype='str')
        parameters_csv = parameters_csv.drop_duplicates(subset=parameters_csv.columns[2:].to_list())
        parameters_csv.to_csv(config['csv_name'], index=False,  float_format='%g')

#     print("============ Accuracy: %.4f%% ==============" % (accuracy * 100.0))
    return accuracy

In [ ]:
def cat_objective(trial):
    global parameters_csv
    cat_params = {
        # "objective" : "Logloss",
        "eval_metric" : "AUC",
        "task_type" : "GPU",
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1), # only CPU
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "used_ram_limit": "12gb",
        # akmeghdad
        'iterations':trial.suggest_int("iterations", 1000, 20000),
        'learning_rate' : trial.suggest_float("learning_rate", 1e-4, 1.0, log=True),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1,15),
        'od_wait':trial.suggest_int('od_wait', 500, 2000),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'verbose': False,
        "random_state": 59,
        
    }
    
    if cat_params["bootstrap_type"] == "Bayesian":
        cat_params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif cat_params["bootstrap_type"] == "Bernoulli":
        cat_params["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    if not GPU :
        del cat_params['task_type']
        cat_params["colsample_bylevel"] = trial.suggest_float("colsample_bylevel", 0.01, 0.1)
    
    model = eval(config['model'])(**cat_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
#         eval_metric=["error", "logloss", "auc"],
        verbose=0,
        early_stopping_rounds = early_stopping
        )
    y_predicted = model.predict_proba(X_test)
    accuracy = roc_auc_score(y_test, y_predicted[:, 1])
    # accuracy = accuracy_score(y_test, y_predicted)
    # print('Accuracy:', round(accuracy_score(y_test, y_predicted[:, 1]) * 100,2),'%')
    
    if accuracy > min_accuracy:
        cat_params['score'] = accuracy
        cat_params['model'] = config['model']
        cat_params['date'] = time.strftime('%Y-%m-%d %H:%M:%S')
        cat_params['task_type'] = 'GPU'
        del cat_params['used_ram_limit']
        
        err,ok1,ok2 = 0,0,0
        id = parameters_csv.shape[0]+1

        for i, k in cat_params.items():
            if i in parameters_csv.columns:
                parameters_csv.loc[id, i] = k
    
        parameters_csv.to_csv(config['csv_name'], index=False,  float_format='%g')
        parameters_csv = pd.read_csv(config['csv_name'], dtype='str')
        parameters_csv = parameters_csv.drop_duplicates(subset=parameters_csv.columns[2:].to_list())
        parameters_csv.to_csv(config['csv_name'], index=False,  float_format='%g')

#     print("============ Accuracy: %.4f%% ==============" % (accuracy * 100.0))
    return accuracy

In [ ]:
def run_study(st):
    time_now = time.time() # 31500 == 8 hr 45 min
    if time_now - time_start < time_limit :
        global study_lst
#         print('**'*10,st,'**'*60,'\n')
        ak_n_trials = 1
        if config['name'] == 'XGB':
            if not os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
                study_lst[st] = optuna.load_study( study_name=competition+"_xgb_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
#                 print('*+*'*60,'\n')
            else:
                study_lst[st] = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_xgb_study", load_if_exists=True)

            study_lst[st].optimize(xgb_objective, n_trials=ak_n_trials)

        elif config['name'] == 'LGBM':

            if os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
                study_lst[st] = optuna.load_study( study_name=competition+"_lgbm_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
#                 print('*+*'*60,'\n')
            else:
                study_lst[st] = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_lgbm_study", load_if_exists=True)
#                 print('*====*'*60,'\n')

            study_lst[st].optimize(lgbm_objective, n_trials=ak_n_trials)
    #         del study_lst[st];gc.collect();

        elif config['name'] == 'CAT':
            if os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
                study_lst[st] = optuna.load_study( study_name=competition+"_cat_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
            else:
                study_lst[st] = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_cat_study", load_if_exists=True)

            study_lst[st].optimize(cat_objective, n_trials=ak_n_trials)
#     else:
#         print('end+++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
# st = 0
# if os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
#     study_lst[st] = optuna.load_study( study_name=competition+"_lgbm_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
#     print('*+*'*60)
# else:
#     study_lst[st] = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_lgbm_study", load_if_exists=True)
#     print('*====*'*60)

# study_lst[st].optimize(lgbm_objective, n_trials=ak_n_trials)

In [ ]:
# %%time
tmp_ak_n_trials = ak_n_trials # hold ak_n_trials for after

if no_stop:
    ak_n_trials = 1
    tmp_ak_n_trials = 1
elif ak_n_trials == 1: 
    tmp_ak_n_trials = 2
else:
    ak_n_trials = 1

if config['name'] == 'DEMO':
    ak_n_trials = 0
    
# if config['name'] == 'XGB':
#     if not os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
#         study_lst = optuna.load_study( study_name=competition+"_xgb_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
#     else:
#         study_lst = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_xgb_study", load_if_exists=True)

# #     study_lst.optimize(xgb_objective, n_trials=ak_n_trials)

# elif config['name'] == 'LGBM':
#     if os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
#         study_lst = optuna.load_study( study_name=competition+"_lgbm_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
#     else:
#         study_lst = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_lgbm_study", load_if_exists=True)

# #     study_lst.optimize(lgbm_objective, n_trials=ak_n_trials)

# elif config['name'] == 'CAT':
#     if os.path.isfile(DATA_PATH + '/working/' + config['db_name']):
#         study_lst = optuna.load_study( study_name=competition+"_cat_study", storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'])
#     else:
#         study_lst = optuna.create_study(direction="maximize",storage="sqlite:////" + DATA_PATH + '/working/' + config['db_name'], study_name=competition+"_cat_study", load_if_exists=True)

# #     study_lst.optimize(cat_objective, n_trials=ak_n_trials)
    
# x = threading.Thread(target=thread_function, args=(1,)) 
# logging.info("Main    : before running thread")
# x.start()
# https://realpython.com/intro-to-python-threading/#producer-consumer-threading

with ThreadPoolExecutor(max_workers=2) as executor:
#     print('|||'*60,'\n')
#     time_now = time.time() # 31500 == 8 hr 45 min
#     if time_now - time_start < time_limit :
#         ak_n_trials = 0 # exit of loop
#         break
#         print('for i','+++++'*60,'\n')
        for i in range(45):
#             time_now = time.time() # 31500 == 8 hr 45 min*
#             print(f'time_now: {time_now} time_all: {time_now - time_start}')
#             if time_now - time_start > time_limit :
#                 break
    #         time.sleep(1)
#             print('executor.submit','|||'*60,'\n')
            executor.submit(run_study, i) 

#     executor.map(run_study, range(45))

        
#     while (ak_n_trials == 10):
#         ak_n_trials = tmp_ak_n_trials # recover ak_n_trials
        
# #         executor.submit(run_study, study_lst) 
# #         executor.map(run_study, range(2))
            
#         time_now = time.time() # 31500 == 8 hr 45 min
#         if time_now - time_start > time_limit :
#             ak_n_trials = 0 # exit of loop